# Testing MCTS with Various Models
Here we will test MCTS first with the DKT model

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from concept_dependency_graph import ConceptDependencyGraph
import data_generator as dg
from student import *
import simple_mdp as sm

In [3]:
n_concepts = 2
use_student2 = True
student2_str = '2' if use_student2 else ''
learn_prob = 0.5
lp_str = '-lp{}'.format(int(learn_prob*100)) if not use_student2 else ''
n_students = 10000
seqlen = 2
filter_mastery = False
filter_str = '' if not filter_mastery else '-filtered'
policy = 'random'
filename = 'test{}-n{}-l{}{}-{}{}.pickle'.format(student2_str, n_students, seqlen,
                                                    lp_str, policy, filter_str)
#concept_tree = sm.create_custom_dependency()
concept_tree = ConceptDependencyGraph()
concept_tree.init_default_tree(n_concepts)
if not use_student2:
    test_student = Student(n=n_concepts,p_trans_satisfied=learn_prob, p_trans_not_satisfied=0.0, p_get_ex_correct_if_concepts_learned=1.0)
else:
    test_student = Student2(n_concepts)
print(filename)

test2-n10000-l2-random.pickle


In [12]:
print ("Initializing synthetic data sets...")
dg.generate_data(concept_tree, student=test_student, n_students=n_students, filter_mastery=filter_mastery, seqlen=seqlen, policy=policy, filename="{}{}".format(dg.SYN_DATA_DIR, filename))
print ("Data generation completed. ")

Initializing synthetic data sets...
Generating data for 10000 students with behavior policy random and sequence length 4.
Data generation completed. 


In [4]:
import dynamics_model_class as dmc
import numpy as np
import dataset_utils
import tensorflow as tf
import tflearn
import copy

In [5]:
# load toy data
data = dataset_utils.load_data(filename='{}{}'.format(dg.SYN_DATA_DIR, filename))

In [6]:
print('Average posttest: {}'.format(sm.expected_reward(data)))
print('Average sparse reward: {}'.format(sm.expected_sparse_reward(data)))
print('Percent of full posttest score: {}'.format(sm.percent_complete(data)))
print('Percent of all seen: {}'.format(sm.percent_all_seen(data)))
input_data_, output_mask_, target_data_ = dataset_utils.preprocess_data_for_rnn(data)

Average posttest: 0.62635
Average sparse reward: 0.2527
Percent of full posttest score: 0.2527
Percent of all seen: 0.7478


In [7]:
train_data = (input_data_[:,:,:], output_mask_[:,:,:], target_data_[:,:,:])
print(input_data_.shape)
print(output_mask_.shape)
print(target_data_.shape)

(10000, 2, 4)
(10000, 2, 2)
(10000, 2, 2)


In [8]:
# test_model hidden=16
# test_model_mid hidden=10
# test_model_small hidden=5
# test_model_tiny hidden=3
model_id = "test2_model2simple_tiny"
dmodel = dmc.DynamicsModel(model_id=model_id, timesteps=seqlen, dropout=1.0, load_checkpoint=False)

Loading RNN dynamics model...
Directory path for tensorboard summaries: ../tensorboard_logs/test2_model2simple_tiny/
Checkpoint directory path: ../checkpoints/test2_model2simple_tiny/
Model loaded.


In [9]:
class ExtractCallback(tflearn.callbacks.Callback):
    def __init__(self):
        self.tstate = None
    def on_epoch_end(self, training_state):
        self.tstate = copy.copy(training_state)
ecall = ExtractCallback()

In [10]:
dmodel.train(train_data, n_epoch=2, callbacks=ecall, load_checkpoint=False)

Training Step: 281  | total loss: 0.04508 | time: 0.975s
| Adam | epoch: 002 | loss: 0.04508 -- iter: 8960/9000
Training Step: 282  | total loss: 0.04504 | time: 1.988s
| Adam | epoch: 002 | loss: 0.04504 | val_loss: 0.04421 -- iter: 9000/9000
--
Type is unsupported, or the types of the items don't match field type in CollectionDef.
'list' object has no attribute 'name'


In [11]:
print(dir(ecall.tstate))
print(ecall.tstate.global_loss)
print(ecall.tstate.val_loss)

['__class__', '__delattr__', '__dict__', '__doc__', '__format__', '__getattribute__', '__hash__', '__init__', '__module__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slotnames__', '__str__', '__subclasshook__', '__weakref__', 'acc_value', 'best_accuracy', 'current_iter', 'epoch', 'global_acc', 'global_loss', 'increaseEpoch', 'increaseStep', 'loss_value', 'resetGlobal', 'step', 'step_time', 'step_time_total', 'update', 'val_acc', 'val_loss']
0.0450353957713
0.0442148507237
